In [ ]:
#| default_exp memory

In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

In [8]:
#| export
import subprocess
import psutil
import tsai.imports as ts

## Check GPU memory usage

In [122]:
#| export
def mem2GB( mem : int) -> int: 
    return round(mem/1024, 2)

def get_decoded_memory(used_total = "used", device = None, print_flag = False):
    if print_flag: print("Get " + used_total + " memory information")
    if device is None:
        if print_flag: print("For all GPUs devices")
        mem_info = subprocess.check_output(["nvidia-smi", "--query-gpu=memory."+used_total, "--format=csv"]).decode("ascii").split("\n")[:-1][1:]
        if print_flag: print("Memory info: ", mem_info)
        mem_values = [int(x.split()[0]) for x in mem_info]
        if print_flag: print("Memory values: ", mem_values)
        return mem_values
    else:
        if print_flag: print("For device", device)
        return int(subprocess.check_output(["nvidia-smi", "--query-gpu=memory."+used_total, "--format=csv,noheader,nounits", "--id=" + str(device)]).decode("ascii").split("\n")[0])
    
def get_gpu_memory(
    device : int = 0, 
    all : bool = False, 
    print_flag : bool = False
):    
    if all:
        if print_flag: print("all")
        total_memory = ts.get_gpu_memory()
        if print_flag: print("--> Get ussed memory info")
        used_memory = get_decoded_memory("used", None, print_flag)
        used_memory = [mem2GB(x) for x in used_memory]
        if print_flag: print("--> Compute array of percentages")
        percentage = [ round((x / y) * 100) for (x, y) in zip(used_memory, total_memory) ]
    else:
        if print_flag: print("one: device ", device)
        total_memory = mem2GB( get_decoded_memory("total", device) )
        used_memory  = mem2GB( get_decoded_memory("used", device) )
        percentage = round((used_memory / total_memory) * 100)

    if print_flag: print("Total ", total_memory, " | Used ", used_memory, " | Percentage ", percentage)
    
    return used_memory, total_memory, percentage

In [123]:
#| hide
print("tsai: ", ts.get_gpu_memory())
print("Final 1 device 0: ", get_gpu_memory(device = 0, all = False, print_flag = True))
print("Final all devices: ", get_gpu_memory(device = None, all = True, print_flag = True))

tsai:  [24.0, 24.0, 24.0]
one: device  0
Total  24.0  | Used  0.0  | Percentage  0
Final 1 device 0:  (0.0, 24.0, 0)
all
--> Get ussed memory info
Get used memory information
For all GPUs devices
Memory info:  ['3 MiB', '782 MiB', '3 MiB']
Memory values:  [3, 782, 3]
--> Compute array of percentages
Total  [24.0, 24.0, 24.0]  | Used  [0.0, 0.76, 0.0]  | Percentage  [0, 3, 0]
Final all devices:  ([0.0, 0.76, 0.0], [24.0, 24.0, 24.0], [0, 3, 0])


In [102]:
#| hide
get_gpu_memory(print_flag = True)
get_gpu_memory(all = True, print_flag = True)
ts.get_gpu_memory()

one
Total  24.0  | Used  0.0  | Percentage  0
all
[782, 3]
--> Compute used_memory_values


AttributeError: 'int' object has no attribute 'split'

In [20]:
#| export
def color_for_percentage(percentage):
    if percentage < 20:
        return "\033[90m"  # Gray
    elif percentage < 40:
        return "\033[94m"  # Blue
    elif percentage < 60:
        return "\033[92m"  # Green
    elif percentage < 80:
        return "\033[93m"  # Orange
    else:
        return "\033[91m"  # Red

In [21]:
#| export
def create_bar(percentage, color_code, length=20):
    filled_length = int(length * percentage // 100)
    bar = "█" * filled_length + "-" * (length - filled_length)
    return color_code + bar + "\033[0m"  # Apply color and reset after bar

In [22]:
#| export
def gpu_memory_status(device=0):
    used, total, percentage = get_gpu_memory(device)
    color_code = color_for_percentage(percentage)
    bar = create_bar(percentage, color_code)
    print(f"GPU | Used mem: {used}")
    print(f"GPU | Used mem: {total}")
    print(f"GPU | Memory Usage: [{bar}] {color_code}{percentage}%\033[0m")

In [26]:
#| hide
gpu_memory_status(all = True)
ts.get_gpu_memory()

TypeError: gpu_memory_status() got an unexpected keyword argument 'all'

In [17]:
#| hide 
ts.my_setup()

os              : Linux-5.15.0-97-generic-x86_64-with-glibc2.31
python          : 3.10.13
tsai            : 0.3.9
fastai          : 2.7.14
fastcore        : 1.5.29
torch           : 1.13.0
device          : 3 gpus (['NVIDIA GeForce RTX 3090', 'NVIDIA GeForce RTX 3090', 'NVIDIA GeForce RTX 3090'])
cpu cores       : 24
threads per cpu : 2
RAM             : 251.54 GB
GPU memory      : [24.0, 24.0, 24.0] GB


## Check CPU memory usage

In [ ]:
#| export
def get_cpu_memory():
    mem = psutil.virtual_memory()
    total_memory = mem.total // (1024**2)  # Convertir a MB
    used_memory = (mem.total - mem.available) // (1024**2)  # Convertir a MB
    percentage = mem.percent
    return used_memory, total_memory, percentage

In [ ]:
#| export
def cpu_memory_status():
    used, total, percentage = get_cpu_memory()
    color_code = color_for_percentage(percentage)
    bar = create_bar(percentage, color_code)
    print(f"CPU | Used mem: {used}")
    print(f"CPU | Used mem: {total}")
    print(f"CPU | Memory Usage: [{bar}] {color_code}{percentage}%\033[0m")

In [ ]:
#| hide
cpu_memory_status()